In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns 
color = sns.color_palette()
import plotly.offline as py
py.init_notebook_mode(connected=True)
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.offline as offline
offline.init_notebook_mode()

from sklearn.model_selection import train_test_split 
import lightgbm as lgb

import cufflinks as cf
cf.go_offline()

import xgboost as xgb 


In [ ]:
#Load the tran and test dataframes 

lp=pd.read_csv("lp.csv")
fb=pd.read.csv("feedback.csv")

application_train = lp.merge(right=fb.reset_index(drop=True), how='inner', on='CUST_ID')
application_train

application_test=lp[(~lp.col1.isin(application_train.CUST_ID))]
application_test

In [ ]:
#Features dataframe using train and test set 
def featuresDF(df_train,df_test,target):
    training=df_train
    testing=df_test

    training['is_train']=1
    training['is_test']=0
    testing['is_train']=0
    testing['is_test']=1
    
    train_X=training.drop([target],axis=1)
    test_X=testing
    data=pd.concat([train_X,test_X],axis=0)
    return data
data=featuresDF(application_train,application_test,"Comment")

In [ ]:
#Target variable extraction 
def target(df_train,target):
    training=df_train
    Y=training[target]
    return Y
    
Y=target(application_train,'TARGET')

In [ ]:
#Function to capture numerical variables 
def _get_numerical_features(df):
    feats=[col for col in list(df.columns) if df[col].dtype!='object']
    return feats 
numFeatures=_get_numerical_features(data)

In [ ]:
#Function to capture categorical variables 
def _get_categorical_features(df):
    feats=[col for col in list(df.columns) if df[col].dtype=='object']
    return feats 
cats=_get_categorical_features(data)

In [ ]:
#Function to factorize categorical variables 
def _factorize_categoricals(df, cats):
    for col in cats:
        df[col], _ = pd.factorize(df[col])
    return df 
#data=_factorize_categoricals(data,cats)

In [ ]:
#Function to convert categorical variables into dummy variables 
def _get_dummies(df,cats):
    for col in cats:
         df = pd.concat([df, pd.get_dummies(df[col], prefix=col)], axis=1)
    return df
#data=_get_dummies(data,cats)

In [ ]:
#Doing factorize or dummy variable encoding 
def encoding(df,flag):
    data_cats = _get_categorical_features(df)
    if(flag==1):
        df = _factorize_categoricals(df,data_cats)
    else:
        df=_get_dummies(df,data_cats)
        df=df.drop(data_cats,axis=1)
    return df
data=encoding(data,0)

In [ ]:
#Missing values Analysis 
def missingValuesAnalysis(data):
    total = data.isnull().sum().sort_values(ascending = False)
    percent = (data.isnull().sum()/data.isnull().count()*100).sort_values(ascending = False)
    return pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
df=missingValuesAnalysis(data)
print(df)

In [ ]:
#Missing values handling 

data1=data.iloc[:,:].values #This only contain values (not headers) . Objective of this is to convet data frame into matrix
#in order to aply imputation and standerdization. 

numInd=_get_numerical_features(data)
catInd=_get_categorical_features(data)

numImputer = Imputer(missing_values ='NaN', strategy='median', axis=0)
numImputer= numImputer.fit(data1[:,(numInd)])
data1[:,(numInd)]=numImputer.transform(data1[:,(numInd)])

catImputer = Imputer(missing_values = 'NaN', strategy = 'most_frequent', axis =0)
catImputer=catImputer.fit(data1[:,(catInd)])
data1[:,(catInd)]=catImputer.transform(data1[:,(catInd)])

#Standerdizing 
min_max_scaler = preprocessing.MinMaxScaler()
data1[:,(numInd)] = min_max_scaler.fit_transform(data1[:,(numInd)])

data=pd.DataFrame(data=data1[0:,0:],columns=data.columns.values)


In [ ]:
#Splitting into train and test set 
ignoreFeatures = ['CUST_ID', 'is_train', 'is_test']
importantFeatures = [col for col in data.columns if col not in ignoreFeatures]
trainX = data[data['is_train'] == 1][importantFeatures]
testX = data[data['is_test'] == 1][importantFeatures]
x_train, x_val, y_train, y_val = train_test_split(trainX, Y, test_size=0.2, random_state=18)

#  Method 1 with train and test split 

In [ ]:
model1 = RandomForestClassifier()
model1=model1.fit(x_train,y_train)    

In [ ]:
predictions1 = model1.predict(x_val)

In [ ]:
print "Train Accuracy :: ", accuracy_score(y_train,model1.predict(x_train))
print "Test Accuracy  :: ", accuracy_score(y_val, predictions1)
print " Confusion matrix ", confusion_matrix(y_val, predictions1)

In [ ]:
#Prediction 
preds1 = model1.predict(testX)
test_id=application_test['CUST_ID']
sub_RF1 = pd.DataFrame()
sub_RF1['CUST_ID'] = test_id
sub_RF1['COMMENT'] = preds1
sub_RF1.to_csv("Prediction_RF1.csv", index=False)
sub_RF1.head()

#  Method 2 with cross validation method-Random search 

In [8]:
from sklearn.ensemble import RandomForestClassifier
from pprint import pprint
from sklearn.model_selection import RandomizedSearchCV
import numpy as np
import pandas as pd

In [5]:
# Look at parameters used by our current forest
model2= RandomForestClassifier(random_state = 42)
print('Parameters currently in use:\n')
pprint(model2.get_params())

Parameters currently in use:

{'bootstrap': True,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 10,
 'n_jobs': 1,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}


In [10]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)

{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}


In [12]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
model2 = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
model2_random = RandomizedSearchCV(estimator = model2, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
model2_random.fit(train_x,train_y)

In [ ]:
#Getting best parameters 
model2_random.best_params_

In [ ]:
#Evaluation of model  
best_random = model2_random.best_estimator_
predictions2 = best_random.predict(x_val)
print "Train Accuracy :: ", accuracy_score(y_train,best_random.predict(x_train))
print "Test Accuracy  :: ", accuracy_score(y_val, predictions2)
print " Confusion matrix ", confusion_matrix(y_val, predictions2)

In [ ]:
#Prediction 
preds2 = best_random.predict(testX)
test_id=application_test['CUST_ID']
sub_RF2 = pd.DataFrame()
sub_RF2['CUST_ID'] = test_id
sub_RF2['COMMENT'] = preds2
sub_RF2.to_csv("Prediction_RF2.csv", index=False)
sub_RF2.head()

# Method 3 with cross validation method-Grid search 

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}
# Create a based model
model3 = RandomForestClassifier()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = model3, param_grid = param_grid, cv = 3, n_jobs = -1, verbose = 2)

In [ ]:
#Getting best parameters 
grid_search.best_params_

In [ ]:
#Evaluation of model  
best_grid = grid_search.best_estimator_
predictions3 = best_grid.predict(x_val)
print "Train Accuracy :: ", accuracy_score(y_train,best_grid.predict(x_train))
print "Test Accuracy  :: ", accuracy_score(y_val, predictions3)
print " Confusion matrix ", confusion_matrix(y_val, predictions3)

In [ ]:
#Prediction 
preds3 = best_grid.predict(testX)
test_id=application_test['CUST_ID']
sub_RF3 = pd.DataFrame()
sub_RF3['CUST_ID'] = test_id
sub_RF3['COMMENT'] = preds3
sub_RF3.to_csv("Prediction_RF3.csv", index=False)
sub_RF3.head()